# Notebook to access to data for the System-level Science Performance Verification Sprint February 3-5 2025

- Confluence page : https://rubinobs.atlassian.net/wiki/spaces/LSSTCOM/pages/372867091/System-level+Science+Performance+Verification+Sprint
- slides : https://docs.google.com/presentation/d/1bPjS5NGtiEL2lfjmvP1UYdG_tMEDfZxX26ayhf7MhtY/edit#slide=id.ga2f7fb1a1f_0_70

- where to find the campains
- https://rubinobs.atlassian.net/wiki/spaces/DM/pages/226656354/LSSTComCam+Intermittent+Cumulative+DRP+Runs

- plot Navigator
- https://usdf-rsp.slac.stanford.edu/plot-navigator
- https://usdf-rsp.slac.stanford.edu/plot-navigator/plot/%2Frepo%2Fmain/LSSTComCam%2Fruns%2FDRP%2FDP1%2Fw_2025_05%2FDM-48666/objectTableCore_coaddInputCount_SkyPlot

- Notebooks examples
- https://github.com/lsst-dm/DMTR-401/blob/main/notebooks/test_LVV-T40_T1240.ipynb
- https://github.com/lsst-dm/DMTR-412/blob/tickets/DM-38728/notebooks/test_LVV-T1751_AM1_AM2.ipynb

In [2]:
# Confirm that the version of the Science Pipelines is recent:
! echo $HOSTNAME
! eups list -s | grep lsst_distrib

dagoret-nb
lsst_distrib          gc7ba34d93f+8892a0c6f9 	current w_2024_50 setup


In [3]:
from lsst.daf.butler import Butler
import lsst.geom as geom

In [4]:
import os
import gc
import glob
import numpy as np
import pandas as pd
import random

import astropy.units as u
from astropy.coordinates import SkyCoord
from astroquery.gaia import Gaia
Gaia.MAIN_GAIA_TABLE = "gaiadr3.gaia_source"  # Reselect Data Release 3, default
Gaia.ROW_LIMIT = 100000
from astropy.visualization import (MinMaxInterval, AsinhStretch, ZScaleInterval, LogStretch, LinearStretch,
                                   ImageNormalize)

In [5]:
# Set plotting defaults
%matplotlib inline
import matplotlib.pyplot as plt
zscale = ZScaleInterval()

# Set up some plotting defaults:
plt.rcParams.update({'figure.figsize' : (12, 8)})
plt.rcParams.update({'font.size' : 24})
plt.rcParams.update({'axes.linewidth' : 3})
plt.rcParams.update({'axes.labelweight' : 3})
plt.rcParams.update({'axes.titleweight' : 5})
plt.rcParams.update({'ytick.major.width' : 3})
plt.rcParams.update({'ytick.minor.width' : 2})
plt.rcParams.update({'ytick.major.size' : 8})
plt.rcParams.update({'ytick.minor.size' : 5})
plt.rcParams.update({'xtick.major.size' : 8})
plt.rcParams.update({'xtick.minor.size' : 5})
plt.rcParams.update({'xtick.major.width' : 3})
plt.rcParams.update({'xtick.minor.width' : 2})
plt.rcParams.update({'xtick.direction' : 'in'})
plt.rcParams.update({'ytick.direction' : 'in'})


In [6]:
# The output repo is tagged with the Jira ticket number "DM-40356":
repo = '/repo/main'
collection = 'LSSTComCam/runs/DRP/DP1/w_2025_05/DM-48666'

In [13]:
# Initialize the butler repo:
butler = Butler(repo, collections=collection)
registry = butler.registry

In [12]:
for _ in registry.queryCollections():
    if "LSSTComCam/runs/DRP/DP1/w_2025_05" in _:
        print(_)

LSSTComCam/runs/DRP/DP1/w_2025_05/DM-48666/20250130T204422Z
LSSTComCam/runs/DRP/DP1/w_2025_05/DM-48666/20250130T225724Z
LSSTComCam/runs/DRP/DP1/w_2025_05/DM-48666/20250130T232326Z
LSSTComCam/runs/DRP/DP1/w_2025_05/DM-48666/20250131T000450Z
LSSTComCam/runs/DRP/DP1/w_2025_05/DM-48666/20250131T013621Z
LSSTComCam/runs/DRP/DP1/w_2025_05/DM-48666/20250131T132156Z
LSSTComCam/runs/DRP/DP1/w_2025_05/DM-48666/20250131T163951Z
LSSTComCam/runs/DRP/DP1/w_2025_05/DM-48666/20250131T165606Z
LSSTComCam/runs/DRP/DP1/w_2025_05/DM-48666/20250131T171816Z
LSSTComCam/runs/DRP/DP1/w_2025_05/DM-48666/20250131T192802Z
LSSTComCam/runs/DRP/DP1/w_2025_05/DM-48666/20250131T180626Z
LSSTComCam/runs/DRP/DP1/w_2025_05/DM-48666/20250202T041533Z
LSSTComCam/runs/DRP/DP1/w_2025_05/DM-48666/20250202T042949Z
LSSTComCam/runs/DRP/DP1/w_2025_05/DM-48666/20250202T063614Z
LSSTComCam/runs/DRP/DP1/w_2025_05/DM-48666/20250202T133020Z
LSSTComCam/runs/DRP/DP1/w_2025_05/DM-48666/20250202T171005Z
LSSTComCam/runs/DRP/DP1/w_2025_05/DM-486

In [15]:
for datasetType in registry.queryDatasetTypes():
    if registry.queryDatasets(datasetType, collections=collection).any(
        execute=False, exact=False
    ):
        # Limit search results to the data products
        if (
            ("_config" not in datasetType.name)
            and ("_log" not in datasetType.name)
            and ("_metadata" not in datasetType.name)
            and ("_resource_usage" not in datasetType.name)
        ):
            print(datasetType)

DatasetType('skyMap', {skymap}, SkyMap)
DatasetType('raw', {band, instrument, day_obs, detector, group, physical_filter, exposure}, Exposure)
DatasetType('camera', {instrument}, Camera, isCalibration=True)
DatasetType('defects', {instrument, detector}, Defects, isCalibration=True)
DatasetType('transmission_optics', {instrument}, TransmissionCurve, isCalibration=True)
DatasetType('transmission_sensor', {instrument, detector}, TransmissionCurve, isCalibration=True)
DatasetType('transmission_filter', {band, instrument, physical_filter}, TransmissionCurve, isCalibration=True)
DatasetType('bias', {instrument, detector}, ExposureF, isCalibration=True)
DatasetType('flat', {band, instrument, detector, physical_filter}, ExposureF, isCalibration=True)
DatasetType('dark', {instrument, detector}, ExposureF, isCalibration=True)
DatasetType('sdss_dr9_fink_v5b', {htm7}, SimpleCatalog)
DatasetType('ps1_pv3_3pi_20170110', {htm7}, SimpleCatalog)
DatasetType('gaia_dr2_20200414', {htm7}, SimpleCatalog)
Da

In [16]:
did_tract2394 = {'instrument':'LSSTComCam', 'tract':2394, 'skymap':'lsst_cells_v1'}
metric_extract2394 = butler.get('matchedVisitCore_metrics', collections=collection, dataId=did_tract2394)

In [17]:
metric_extract2394 

{'stellarAstrometricRepeatability1': [Measurement('u_AM1', <Quantity 17.33343272 mas>), Measurement('u_AF1', <Quantity 27.76412776 %>), Measurement('u_AD1', <Quantity 32.51939029 mas>), Measurement('g_AM1', <Quantity 14.9082109 mas>), Measurement('g_AF1', <Quantity 27.28738152 %>), Measurement('g_AD1', <Quantity 32.8520434 mas>), Measurement('r_AM1', <Quantity 13.01244702 mas>), Measurement('r_AF1', <Quantity 13.65845067 %>), Measurement('r_AD1', <Quantity 22.47752361 mas>), Measurement('i_AM1', <Quantity 10.73313631 mas>), Measurement('i_AF1', <Quantity 7.925028 %>), Measurement('i_AD1', <Quantity 18.53151446 mas>), Measurement('z_AM1', <Quantity 13.37065486 mas>), Measurement('z_AF1', <Quantity 14.90318724 %>), Measurement('z_AD1', <Quantity 23.18498579 mas>), Measurement('y_AM1', <Quantity 11.48309677 mas>), Measurement('y_AF1', <Quantity 9.35112722 %>), Measurement('y_AD1', <Quantity 19.5892845 mas>)], 'stellarAstrometricRepeatability2': [Measurement('u_AM2', <Quantity 16.71457986 

In [18]:
print('Tract 2394:\n')
for met in metric_extract2394['stellarAstrometricRepeatability1']:
    if ('AM1' in met.metric_name.metric) and \
       (met.metric_name.metric.startswith('g_') or met.metric_name.metric.startswith('r_') or \
        met.metric_name.metric.startswith('i_')):
        print(met.datum)
print('\n')
for met in metric_extract2394['stellarAstrometricRepeatability2']:
    if ('AM2' in met.metric_name.metric) and \
       (met.metric_name.metric.startswith('g_') or met.metric_name.metric.startswith('r_') or \
        met.metric_name.metric.startswith('i_')):
        print(met.datum)



Tract 2394:

g_AM1 = 14.908210900360078 mas
r_AM1 = 13.012447016861413 mas
i_AM1 = 10.733136314524783 mas


g_AM2 = 14.033573691280074 mas
r_AM2 = 12.377473604825562 mas
i_AM2 = 10.132747414789971 mas
